init

In [1]:
from pymystem3 import Mystem
my_stem = Mystem()

import nltk

try:
    nltk.corpus.stopwords.words('english')
except:
    nltk.download('stopwords')

import pandas as pd
from collections import defaultdict

import gensim.models

DATA_PATH = '/data/'
ROW_CAPTION_PATH = DATA_PATH + 'row_caption/'
CAPTION_PATH = DATA_PATH + 'caption/'
LEM_CAPTION_PATH = DATA_PATH + 'lem_caption/'
CAPTIONS = ['posts2016.csv', 'posts2017.csv', 'posts2018.csv', 'posts2019.csv', 'posts2020.csv']
USE_STOPWORDS = False

if not USE_STOPWORDS:
    CAPTION_PATH = DATA_PATH + 'caption_full/'
    LEM_CAPTION_PATH = DATA_PATH + 'lem_caption_full/'

print('success')

success


In [2]:
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_non_alphanum

MIN_DOCUMENT_SIZE = 2
HASH_DELIMITER = 'ЙWЙ'

WITH_STOPWORDS = False

useless_symbols = set(['_', 'ー'])

filter_words = set()
if USE_STOPWORDS:
    # load stopwords from nltk
    filter_words = filter_words.union(set(stopwords.words('english')))
    filter_words = filter_words.union(set(stopwords.words('russian')))
    # load own stopwords
    with open('stopwords_full.txt') as f:
        filter_words = filter_words.union(set([word for line in f for word in line.split()]))


# input: document: str
# output: document: str
# make preprocessing of documents, remove useless symbols, remove useless words
def preprocessor(s: str):
    # all letter to lower case
    s = s.lower() 
    
    # save '#' symbol
    # s = s.replace('#', HASH_DELIMITER)
    
    # remove all symbol, which aren't letter or num
    s = strip_non_alphanum(s)
    # strip_non_alphanum don't remove '_' symbol
    s = ''.join(map(lambda c: ' ' if c in useless_symbols else c, s))

    # restore '#' symbol
    # s = s.replace(HASH_DELIMITER, ' #')
  
    #s = ''.join(my_stem.lemmatize(s))
    # replace all space symbol like space, tab, \n to simple space and remove double and for space
    s = strip_multiple_whitespaces(s)
    # remove empty words and words from filtered words 
    s = ' '.join(list(filter(lambda w:  len(w) > 0 and not (w in filter_words or w.isdigit()), s.split(' '))))
    return s


for post_file in CAPTIONS:
    # load file and drop documents with empty text
    df = pd.read_csv(ROW_CAPTION_PATH + post_file).dropna()
    # preprocess of corpus
    df['caption'] = df['caption'].apply(preprocessor)
    # remove empty documents
    df = df[df['caption'].map(len) > MIN_DOCUMENT_SIZE]
    # save results
    df.to_csv(r'' + CAPTION_PATH + post_file, index=False)
    print("finished " + post_file)

finished posts2016.csv
finished posts2017.csv
finished posts2018.csv
finished posts2019.csv
finished posts2020.csv


Вместо леммитицазии всех докуентов поотдельности можно обработать весь текст, после составить словарь всех слов, леммитизировать все по одному разу и перезаписать текст постов

* pymystem3 - not very fast
* spacy - need to check
* pymorphy2 - slow

In [4]:
# fast lemmatize
from collections import defaultdict

d = defaultdict(int)

# load all words to dictionary
for i in range(len(CAPTIONS)):
    df = pd.read_csv(CAPTION_PATH + CAPTIONS[i])

    for text in df['caption']:
        words = text.split(' ')
        for word in words:
            d[word] += 1
    
    print('load words from ' + CAPTIONS[i])


# create lemmatized dictionary
d_lem = { word: ''.join(my_stem.lemmatize(word)[:-1]) for word in d } # [:-1] for remove \n after lemmatize

# save dictionary 
f = open(DATA_PATH + 'dictionary.txt', 'w')
for word in d:
    f.write(word + ',' + d_lem[word] + ',' + str(d[word]) + '\n')
f.close()

# replece all words in dataset to lemmatized words
for i in range(len(CAPTIONS)):
    df = pd.read_csv(CAPTION_PATH + CAPTIONS[i])

    df['caption'] = df['caption'].apply(lambda caption: ' '.join(list(map(lambda word: d_lem[word], caption.split(' ')))))

    df.to_csv(r'' + LEM_CAPTION_PATH + CAPTIONS[i], index=False)
    print("repleced words in " + CAPTIONS[i])



load words from posts2016.csv
load words from posts2017.csv
load words from posts2018.csv
load words from posts2019.csv
load words from posts2020.csv
repleced words in posts2016.csv
repleced words in posts2017.csv
repleced words in posts2018.csv
repleced words in posts2019.csv
repleced words in posts2020.csv


In [ ]:
with open(DATA_PATH + 'dictionary.txt') as f:
    d_list = [[token for token in line.split(',')] for line in f]


d_lem = defaultdict(int)


for note in d_list:
    d_lem[note[1]] += int(note[2])

l = list(d_lem.items())   
l.sort(reverse=True, key=lambda item: item[1])

print(len(l))
l = list(filter(lambda item: item[1] > 1, l))
print(len(l))

for i in range(500):
    print(i, l[i])
